In [1]:
import numpy as np
import pandas as pd
import talib

In [2]:
import random
random.seed(42)

In [3]:
dataset = pd.read_csv('data/RELIANCE.NS.csv')
dataset = dataset.dropna()
dataset = dataset[['Open', 'High', 'Low', 'Close']]

KeyError: "['Open' 'High' 'Low' 'Close'] not in index"